References :<br>
https://www.youtube.com/watch?v=6zm9NC9uRkk <br>
https://github.com/skipgram/modern-nlp-in-python-2019/blob/master/notebooks/Modern_NLP_in_Python.ipynb

In [1]:
import os
data_directory = os.path.join('.','yelp_dataset')

businesses_filepath = os.path.join(data_directory, 'business.json')

with open(businesses_filepath) as f:
    first_business_record = f.readline() 
    
print(first_business_record)

{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}



In [2]:
#Reviews

review_path = os.path.join(data_directory,'review.json')

with open(review_path) as f:
    first_review = f.readline()
    
print(first_review)

{"review_id":"xQY8N_XvtGbearJ5X4QryQ","user_id":"OwjRMXRC0KyPrIlcjaXeFQ","business_id":"-MhfebM0QIsKt87iDN-FNw","stars":2.0,"useful":5,"funny":0,"cool":0,"text":"As someone who has worked with many museums, I was eager to visit this gallery on my most recent trip to Las Vegas. When I saw they would be showing infamous eggs of the House of Faberge from the Virginia Museum of Fine Arts (VMFA), I knew I had to go!\n\nTucked away near the gelateria and the garden, the Gallery is pretty much hidden from view. It's what real estate agents would call \"cozy\" or \"charming\" - basically any euphemism for small.\n\nThat being said, you can still see wonderful art at a gallery of any size, so why the two *s you ask? Let me tell you:\n\n* pricing for this, while relatively inexpensive for a Las Vegas attraction, is completely over the top. For the space and the amount of art you can fit in there, it is a bit much.\n* it's not kid friendly at all. Seriously, don't bring them.\n* the security is n

In [3]:
#Extract all business ids
import json

restaurant_ids=set()

with open(businesses_filepath) as f:
    
    for business_record in f:
        business = json.loads(business_record)
        
        if not business.get('categories'):
            continue
        
        if 'Restaurants' not in business['categories']:
            continue
            

        restaurant_ids.add(business['business_id'])
        
restaurant_ids = frozenset(restaurant_ids)

print(f'{len(restaurant_ids)} Restaurants in Dataset')

63944 Restaurants in Dataset


In [4]:
#Create a cratch directory to store all reviews
scratch_path = os.path.join('.','scratch')
try:
    os.mkdir(scratch_path)
except FileExistsError:
    pass

review_txt_filepath = os.path.join(scratch_path,'review_text_all.txt')

In [5]:
#Extract all reviews snd store it in the new file created

execute = False

review_count = 0

if execute:
    
    with open(review_txt_filepath,"w") as review_txt_file:
        
        with open(review_path) as review_file:
            
            for review_json in review_file:
                review = json.loads(review_json)
                
                if review['business_id'] not in restaurant_ids:
                    continue
                    
                review_txt_file.write(review['text'].replace("\n","\\n")+"\n")
                review_count+=1
                
    print(f'Text from {review_count:,} restaurant reviews written to the new txt file.')

else:
    
    # count the reviews in the existing file
    with open(review_txt_filepath) as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(f'Text from {review_count + 1:,} restaurant reviews in the txt file.')
                
                


Text from 5,058,162 restaurant reviews in the txt file.


In [6]:
import spacy
from spacy import displacy
import pandas as pd
import itertools as it

nlp = spacy.load('en_core_web_sm')

In [7]:
#Length of each review
review_dict={}
with open(review_txt_filepath) as review_txt_file:
    review_dict = {review_count:len(line) for review_count,line in enumerate(review_txt_file)}
        
    

In [9]:
max_key = max(review_dict,key=review_dict.get)
print(max_key)

1788257


In [58]:
#Sample Review
#10 14 15 22
review_num = 22
with open(review_txt_filepath) as f:
    sample_review = list(it.islice(f, review_num, review_num+1))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

This actually used to be one of my favorite hotel/casinos back in the day.  Oh, how things have changed.

There are some goods, however.  If you are a conventioneer, you have instant and direct walking access to the main convention center.  But don't think about parking here for said conventions.  $30 a day for parking, Hilton?  Seriously?  That's robbery.  And greed.

The biggest disappointment is the killing off of Star Trek: The Experience over two years ago.  Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  It would be one thing if they had something in mind to replace it, but either poor planning or the economy.... It's dead, Jim. 

The "Space Lounge" is still there.  An entire casino area with a heavy sci-fi theme to it.  Except there's no casino here.  It's a huge space themed area with a ghost town bar, no slots, a few couches, a makeshift stage, beer signs, and a few remnants of Star Trek themes.  All of 

In [59]:
%%time
parsed_view = nlp(sample_review)

CPU times: user 267 ms, sys: 25.2 ms, total: 292 ms
Wall time: 160 ms


In [60]:
parsed_view = nlp(sample_review)

In [61]:
print(parsed_view)

This actually used to be one of my favorite hotel/casinos back in the day.  Oh, how things have changed.

There are some goods, however.  If you are a conventioneer, you have instant and direct walking access to the main convention center.  But don't think about parking here for said conventions.  $30 a day for parking, Hilton?  Seriously?  That's robbery.  And greed.

The biggest disappointment is the killing off of Star Trek: The Experience over two years ago.  Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  It would be one thing if they had something in mind to replace it, but either poor planning or the economy.... It's dead, Jim. 

The "Space Lounge" is still there.  An entire casino area with a heavy sci-fi theme to it.  Except there's no casino here.  It's a huge space themed area with a ghost town bar, no slots, a few couches, a makeshift stage, beer signs, and a few remnants of Star Trek themes.  All of 

In [63]:
for num,sentence in enumerate(parsed_view.sents):
    print(f'Sentence {num+1}')
    print(sentence)
    print('')

Sentence 1
This actually used to be one of my favorite hotel/casinos back in the day.  

Sentence 2
Oh, how things have changed.



Sentence 3
There are some goods, however.  

Sentence 4
If you are a conventioneer, you have instant and direct walking access to the main convention center.  

Sentence 5
But don't think about parking here for said conventions.  

Sentence 6
$30 a day for parking, Hilton?  

Sentence 7
Seriously?  

Sentence 8
That's robbery.  

Sentence 9
And greed.



Sentence 10
The biggest disappointment is the killing off of Star Trek:

Sentence 11
The Experience over two years ago.  

Sentence 12
Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  

Sentence 13
It would be one thing if they had something in mind to replace it, but either poor planning or the economy....

Sentence 14
It's dead, Jim. 



Sentence 15
The "Space Lounge" is still there.  

Sentence 16
An entire casino area with a heavy

### Text Normalization

In [64]:
token_text = [token.orth_ for token in parsed_view]
token_lemma = [token.lemma_ for token in parsed_view]
token_shape = [token.shape_ for token in parsed_view]

pd.DataFrame(zip(token_text,token_lemma,token_shape),columns=["token_text","token_lemma","token_shape"])

,token_text,token_lemma,token_shape
0,This,this,Xxxx
1,actually,actually,xxxx
2,used,use,xxxx
3,to,to,xx
4,be,be,xx
5,one,one,xxx
6,of,of,xx
7,my,-PRON-,xx
8,favorite,favorite,xxxx
9,hotel,hotel,xxxx


### Part of Speech

In [65]:
token_pos = [token.pos_ for token in parsed_view]
pd.DataFrame(zip(token_text,token_pos),columns=["Text","POS"])

,Text,POS
0,This,DET
1,actually,ADV
2,used,VERB
3,to,PART
4,be,AUX
5,one,NUM
6,of,ADP
7,my,DET
8,favorite,ADJ
9,hotel,NOUN


In [66]:
displacy.render(parsed_view,style="ent")

In [68]:
for num, entity in enumerate(parsed_view.ents):
    print(f'Entity {num + 1}:', entity, '-', entity.label_)
    print('')

Entity 1: the day - DATE

Entity 2: 30 - MONEY

Entity 3: Hilton - ORG

Entity 4: Star Trek: - WORK_OF_ART

Entity 5: two years ago - DATE

Entity 6: Jim - PERSON

Entity 7: Star Trek - WORK_OF_ART

Entity 8: Couches - PERSON

Entity 9: Benihana - PERSON

Entity 10: Vegas - GPE

Entity 11: Hilton - ORG

Entity 12: Pizza Hut Express - ORG

Entity 13: Benihana - PRODUCT

Entity 14: Star Trek - WORK_OF_ART

Entity 15: two-plus years - DATE

Entity 16: first - ORDINAL

Entity 17: Hilton - ORG

Entity 18: one - CARDINAL

Entity 19: Vegas - GPE



In [69]:
token_entity_type = [token.ent_type_ for token in parsed_view]
token_entity_iob = [token.ent_iob_ for token in parsed_view]

pd.DataFrame(
    zip(token_text, token_entity_type, token_entity_iob),
    columns=['token_text', 'entity_type', 'inside_outside_begin']
    )

,token_text,entity_type,inside_outside_begin
0,This,,O
1,actually,,O
2,used,,O
3,to,,O
4,be,,O
5,one,,O
6,of,,O
7,my,,O
8,favorite,,O
9,hotel,,O


In [70]:
token_attributes = [
    (
        token.orth_,
        token.prob,
        token.is_stop,
        token.is_punct,
        token.is_space,
        token.like_num,
        token.is_oov,
        
    )for token in parsed_view
    
]

df = pd.DataFrame(token_attributes,columns = [
    'text',
    'log_probability',
    'stop?',
    'punctuation?',
    'space',
    'number',
    'out of vocabulary']
    )

df.loc[:,'stop?':'out of vocabulary'] = df.loc[:,'stop?':'out of vocabulary'].applymap(lambda x : x if x else "")
df

,text,log_probability,stop?,punctuation?,space,number,out of vocabulary
0,This,-20.0,True,,,,True
1,actually,-20.0,,,,,True
2,used,-20.0,True,,,,True
3,to,-20.0,True,,,,True
4,be,-20.0,True,,,,True
5,one,-20.0,True,,,True,True
6,of,-20.0,True,,,,True
7,my,-20.0,True,,,,True
8,favorite,-20.0,,,,,True
9,hotel,-20.0,,,,,True


## Phrase Modelling

In [71]:
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import LineSentence

In [158]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def pronoun_lemmatize(token):
    """
    helper function to preserve pronouns and force lowercasing while lemmatizing
    """
    
    if token.lemma_ == '-PRON-':
        return token.lower_
    
    else:
        return token.lemma_.lower()

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with open(filename) as f:
        for review in f:
            yield review.replace('\\n', '\n')
    

We need to reiterate over 5Million reviews and it would take 12-14 Hours. 
<br><br>
Let's create a sample dataset out of reviews_text_all.txt

**For 100,000 Reviews it took 17min** <br>
CPU times: user 43min 46s, sys: 4min, total: 47min 46s <br>
Wall time: 16min 59s

In [159]:
review_txt_filepath_sample_path = os.path.join(scratch_path,'review_txt_file_sample.txt')



In [165]:
reviews_to_copy = 100_000
counter=0
with open(review_txt_filepath_sample_path,'w') as review_txt_filepath_sample:
    with open(review_txt_filepath) as f:
        for review_line in f:
            if counter < reviews_to_copy:
                review_txt_filepath_sample.write(review_line)
                counter+=1

In [86]:
#Let's Go

In [166]:
review_lemmatized_filepath = os.path.join(scratch_path, 'review_lemmatized_all.txt')
sentence_lemmatized_filepath = os.path.join(scratch_path, 'sentence_lemmatized_all.txt')

In [168]:
%%time
execute = False

if execute:
    
    with open(review_lemmatized_filepath,'w') as review_lemmatized_file:
        with open(sentence_lemmatized_filepath,'w') as sentence_lemmatized_file:
            
            pipe = nlp.pipe(
                line_review(review_txt_filepath_sample_path),
                batch_size=1000,
                n_threads=8
            )
            
            for parsed_review in pipe:
                
                lemmatized_review = ' '.join([pronoun_lemmatize(token) for token in parsed_review if not punct_space(token)])
                
                review_lemmatized_file.write(lemmatized_review+'\n')
                
                for sentence in parsed_review.sents:
                    lemmatized_sentence = ' '.join([pronoun_lemmatize(token) for token in sentence if not punct_space(token)])
                    
                    sentence_lemmatized_file.write(lemmatized_sentence+"\n")

   

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 7.87 µs


In [169]:
#gensim's LineSentence class provides a convenient iterator for working with other gensim components.
sentences_unigrams = LineSentence(sentence_lemmatized_filepath)

In [171]:
#Sample

for sentence_unigrams in it.islice(sentences_unigrams,60,70):
    print(' '.join(sentence_unigrams))
    print()

and i hope they continue to do well

order

in delivery review

i discover this place from the front desk of the airport hotel and they recommend it highly

i call to place my order for delivery to my room

they be very accommodating

won

ton soup nice broth and tender win ton

chicken with diced vegetable and cashew again fresh vegetable be use and good sized piece of chicken

crispy beef much like a ginger beef nice serving of beef with a sweet chilli sauce



### Bigram

In [172]:
bigram_model_filepath = os.path.join(scratch_path, 'bigram_phrase_model')

In [173]:
execute = False

if execute:

    bigram_phrases = Phrases(sentences_unigrams)
    
    # Turn the finished Phrases model into a "Phraser" object,
    # which is optimized for speed and memory use
    bigram_phrases = Phraser(bigram_phrases)
    bigram_phrases.save(bigram_model_filepath)

In [174]:
# Load the finished model

bigram_phrases = Phraser.load(bigram_model_filepath)

In [175]:
sentences_bigrams_filepath = os.path.join(scratch_path,'sentence_bigram_phrases_all.txt')

In [177]:
execute = False

if execute:
    with open(sentences_bigrams_filepath,'w') as f:
        
        for sentence_unigrams in sentences_unigrams:
            sentence_bigrams = ' '.join(bigram_phrases[sentence_unigrams])
            f.write(sentence_bigrams+'\n')

In [178]:
sentences_bigrams = LineSentence(sentences_bigrams_filepath)

In [179]:
for sentence_bigrams in it.islice(sentences_bigrams, 60, 70):
    print(' '.join(sentence_bigrams))
    print('')

and i hope they continue to do well

order

in delivery review

i discover this place from the front_desk of the airport hotel and they recommend it highly

i call to place my order for delivery to my room

they be very_accommodating

won

ton soup nice broth and tender win_ton

chicken with diced vegetable and cashew again fresh vegetable be use and good sized piece of chicken

crispy beef much like a ginger beef nice serving of beef with a sweet chilli sauce



In [180]:
trigram_model_filepath = os.path.join(scratch_path, 'trigram_phrase_model')

In [182]:
# this is a bit time consuming - set execute = True
# if you want to execute modeling yourself.

execute = False

if execute:

    trigram_phrases = Phrases(sentences_bigrams)
    
    # Turn the finished Phrases model into a "Phraser" object,
    # which is optimized for speed and memory use
    trigram_phrases = Phraser(trigram_phrases)
    trigram_phrases.save(trigram_model_filepath)

In [183]:
# load the finished model from disk
trigram_phrases = Phraser.load(trigram_model_filepath)

In [184]:
sentences_trigrams_filepath = os.path.join(scratch_path, 'sentence_trigram_phrases_all.txt')

In [186]:
# this is a bit time consuming - set execute = True
# if you want to execute data prep yourself.

execute = False

if execute:

    with open(sentences_trigrams_filepath, 'w') as f:
        
        for sentence_bigrams in sentences_bigrams:
            
            sentence_trigrams = ' '.join(trigram_phrases[sentence_bigrams])
            
            f.write(sentence_trigrams + '\n')

In [187]:
sentences_trigrams = LineSentence(sentences_trigrams_filepath)

In [195]:
for sentence_trigrams in it.islice(sentences_trigrams, 20, 30):
    print(' '.join(sentence_trigrams))
    print('')

my_only_gripe be the one fella with red hair

he make the round while clean but no smile or personality a few night_ago he toss the napkin i just put on the table to help go with my meal

after i be do he just reach for my tray no_excuse me or be you do with that

i realize he be try to do his job quickly but a little table manner go along way

that be say still like to grub here and glad that there be finally a cane_'s close to me

this be definitely my_favorite fast food sub_shop

ingredient be everything and everything i see and taste here tell me that they be use top grade fresh_ingredient

the brisket sandwich be probably my_favorite and it be the one my_wife always get

unlike her i often bounce around the menu to try different thing

definitely a step_up from subway quiznos jimmy_johns etc in my_opinion



In [190]:
review_trigrams_filepath = os.path.join(scratch_path, 'review_trigrams_all.txt')

In [192]:

execute = False

if execute:
    
    reviews_lemmatized = LineSentence(review_lemmatized_filepath)

    with open(review_trigrams_filepath, 'w') as f:
        
        for review_unigrams in reviews_lemmatized:
                        
            # apply the first-order and second-order phrase models
            review_bigrams = bigram_phrases[review_unigrams]
            review_trigrams = trigram_phrases[review_bigrams]

            # remove any remaining stopwords
            review_trigrams = [
                term
                for term in review_trigrams
                if term not in nlp.Defaults.stop_words
                ]

            # write the transformed review as a line in the new file
            review_trigrams = ' '.join(review_trigrams)
            f.write(review_trigrams + '\n')

In [194]:
review_num = 22

print('Original:' + '\n')

for review in it.islice(line_review(review_txt_filepath), review_num, review_num+1):
    print(review)

print('----' + '\n')
print('Transformed:' + '\n')

with open(review_trigrams_filepath) as f:
    for review in it.islice(f, review_num, review_num+1):
        print(review)

Original:

This actually used to be one of my favorite hotel/casinos back in the day.  Oh, how things have changed.

There are some goods, however.  If you are a conventioneer, you have instant and direct walking access to the main convention center.  But don't think about parking here for said conventions.  $30 a day for parking, Hilton?  Seriously?  That's robbery.  And greed.

The biggest disappointment is the killing off of Star Trek: The Experience over two years ago.  Despite this place being off-Strip, I made it a point to go here because they had such a fun and unique attraction here.  It would be one thing if they had something in mind to replace it, but either poor planning or the economy.... It's dead, Jim. 

The "Space Lounge" is still there.  An entire casino area with a heavy sci-fi theme to it.  Except there's no casino here.  It's a huge space themed area with a ghost town bar, no slots, a few couches, a makeshift stage, beer signs, and a few remnants of Star Trek theme

## Topic Modelling using LDA

In [199]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [202]:
dictionary_filepath = os.path.join(scratch_path,'trigram_dict_all.dict')

In [203]:
execute = False

if execute:

    reviews_trigrams = LineSentence(review_trigrams_filepath)

    # learn the dictionary by iterating over all of the reviews
    dictionary_trigrams = Dictionary(reviews_trigrams)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    dictionary_trigrams.filter_extremes(no_below=20, no_above=0.4)
    dictionary_trigrams.compactify()

    dictionary_trigrams.save(dictionary_filepath)

In [204]:
dictionary_trigrams = Dictionary.load(dictionary_filepath)

## Bag of Words

In [205]:
bow_corpus_filepath = os.path.join(scratch_path,'bow_trigrams_corpus_all.mm')

In [206]:
def bow_generator(filepath):
    
    for review in LineSentence(filepath):
        yield dictionary_trigrams.doc2bow(review)

In [208]:
execute = False

if execute:
    MmCorpus.serialize(
        bow_corpus_filepath,
        bow_generator(review_trigrams_filepath)
    )

In [209]:
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(bow_corpus_filepath)

In [211]:
lda_model_filepath = os.path.join(scratch_path, 'lda_model_all')

In [213]:
execute = False

if execute:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
    lda = LdaMulticore(
        trigram_bow_corpus,
        num_topics=50,
        id2word=dictionary_trigrams,
        workers=7
    )
    
    lda.save(lda_model_filepath)

In [214]:
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

In [215]:
def explore_topic(topic_number, topn=25):
        
    print(f'{"term":20} {"frequency"}' + '\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(f'{term:20} {round(frequency, 3):.3f}')

In [225]:
explore_topic(topic_number=20)

term                 frequency

great                0.021
time                 0.016
come                 0.016
service              0.016
price                0.011
little               0.011
love                 0.010
delicious            0.010
staff                0.009
nice                 0.009
try                  0.008
eat                  0.007
order                0.006
restaurant           0.006
happy_hour           0.006
lunch                0.006
like                 0.006
amazing              0.006
friendly             0.005
definitely           0.005
recommend            0.005
wait                 0.005
decent               0.005
2                    0.005
location             0.005
